# 네이버 블로그 게시물 리스트 수집

## 필요한 라이브러리

In [ ]:
# !pip install selenium
# !pip install pyperclip

In [ ]:
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pyperclip

import time
from tqdm import tqdm

import re

import os.path
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
import json


## 게시물 리스트 한 페이지 수집(requests)

In [ ]:
blogId ='블로그ID'
currentPage= 1
categoryNo= 0
# categoryNo= 0 : 전체카테고리

def category_list(blogId: 'str', currentPage: 'int'=1, categoryNo: 'int'=0) -> list:
    
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
}
    response = requests.get(f"https://blog.naver.com/PostTitleListAsync.naver?blogId={blogId}&viewdate=&currentPage={currentPage}&categoryNo={categoryNo}&parentCategoryNo=&countPerPage=30", headers=headers)
    soup = bs(response.text, 'html.parser')
    category_list = json.loads(soup.text)

    return category_list['postList']

## 게시물 리스트 모든 페이지 수집(requests)

In [ ]:
def all_category_list(blogId: 'str', categoryNo: 'int'=0):
    
    category_list_old = pd.DataFrame()
    category_list_new = pd.DataFrame()
    category_list_df = pd.DataFrame()
    currentPage = 1
    end = False
    while  (end == False):
        if(currentPage == 1):
            category_list_new = category_list(blogId, currentPage, categoryNo)
            category_list_df = pd.DataFrame(category_list_new)
        else:
            category_list_new = category_list(blogId, currentPage, categoryNo)
            if(category_list_old == category_list_new):
                end = True
            else:
                category_list_df = pd.concat([category_list_df,  pd.DataFrame(category_list_new)])
        category_list_old = category_list_new
        currentPage += 1
    category_list_df = category_list_df.reset_index(drop=True)
    category_list_df.to_csv(f'category_list({blogId}_{categoryNo}).csv', index=False)
    return category_list_df

## category_list 사용 예시

In [ ]:
category_list(blogId)

[{'sellerServiceStatus': '',
  'logNo': '222847596106',
  'title': '%EC%A0%95%EC%A7%80%EC%9A%B0+%EC%9E%91%EA%B0%80%EB%8B%98+%EB%B6%81%ED%86%A0%ED%81%AC',
  'categoryNo': '25',
  'parentCategoryNo': '34',
  'sourceCode': '0',
  'commentCount': '1',
  'readCount': '',
  'addDate': '2022. 8. 13.',
  'openType': '2',
  'searchYn': 'true',
  'greenReviewBannerYn': False,
  'memologMovingYn': '0',
  'isPostSelectable': '0',
  'isPostNotOpen': '0',
  'isPostBlocked': 0,
  'isBlockTmpForced': 0,
  'postProductStatus': ''},
 {'sellerServiceStatus': '',
  'logNo': '222847537783',
  'title': '%EB%88%85%EB%88%85',
  'categoryNo': '24',
  'parentCategoryNo': '34',
  'sourceCode': '0',
  'commentCount': '',
  'readCount': '',
  'addDate': '2022. 8. 13.',
  'openType': '2',
  'searchYn': 'true',
  'greenReviewBannerYn': False,
  'memologMovingYn': '0',
  'isPostSelectable': '0',
  'isPostNotOpen': '0',
  'isPostBlocked': 0,
  'isBlockTmpForced': 0,
  'postProductStatus': ''},
 {'sellerServiceStatus':

## all_category_list 사용예시

In [ ]:
category_list_df = all_category_list(blogId, categoryNo)

In [ ]:
category_list_df

,sellerServiceStatus,logNo,title,categoryNo,parentCategoryNo,sourceCode,commentCount,readCount,addDate,openType,searchYn,greenReviewBannerYn,memologMovingYn,isPostSelectable,isPostNotOpen,isPostBlocked,isBlockTmpForced,postProductStatus
0,,222847537783,%EB%88%85%EB%88%85,24,34,0,,,2022. 8. 13.,2,true,False,0,0,0,0,0,
1,,222843158417,%EB%B9%84%EB%A5%BC+%EB%A7%9E%EA%B3%A0+%EC%8B%B...,24,34,0,,,2022. 8. 8.,2,true,False,0,0,0,0,0,
2,,222835152819,%EC%9D%BD%EC%9D%84+%EA%B2%83%EB%93%A4%EC%9D%80...,24,34,0,1,,2022. 7. 31.,2,true,False,0,0,0,0,0,
3,,222834974635,%ED%95%9C%EC%97%AC%EB%A6%84%EC%9D%98+%EB%AC%B4...,24,34,0,,,2022. 7. 31.,2,true,False,0,0,0,0,0,
4,,222834965199,%5B%ED%9B%84%EA%B8%B0%5D+%EC%9B%94%EA%B8%89%EC...,24,34,0,,,2022. 7. 31.,2,true,False,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,,110156306102,%EB%82%B4+%EC%9D%B4+%EB%A6%84+,24,34,0,,,2013. 1. 10.,2,false,False,0,0,0,0,0,
351,,110156150095,%EC%B1%85+%EA%B0%88+%ED%94%BC+,24,34,0,,,2013. 1. 9.,2,false,False,0,0,0,0,0,
352,,110155822053,%26%2339%3B%EC%A2%8B%EC%95%84%EC%9A%94%26%2339...,24,34,0,1,,2013. 1. 5.,2,true,False,0,0,0,0,0,
353,,110155735413,%26%238226%3B_%26%238226%3B,24,34,0,1,,2013. 1. 3.,2,true,False,0,0,0,0,0,


## 게시물 리스트 전체 페이지 수집(selenium)
* 로그인 후, 블로그 서로이웃 일시 서로이웃 게시물까지 가져옵니다.

In [ ]:
blogId = '블로그ID'
currentPage = 1
categoryNo = 0

# 0-1. 브라우저 안띄우기
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)

# 0-2. 브라우저 띄우기
browser = webdriver.Chrome('chromedriver.exe') # 현재파일과 동일한 경로일 경우 생략 가능

user_id = '사용자ID'
user_pw = '사용자PW'

# 1. 네이버 이동
browser.get('http://naver.com')

# 2. 로그인 버튼 클릭
elem = browser.find_element_by_class_name('link_login')
elem.click()

# 3. id 복사 붙여넣기
elem_id = browser.find_element_by_id('id')
elem_id.click()
pyperclip.copy(user_id)
elem_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 4. pw 복사 붙여넣기
elem_pw = browser.find_element_by_id('pw')
elem_pw.click()
pyperclip.copy(user_pw)
elem_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 5-1. 로그인 버튼 클릭
browser.find_element_by_id('log.login').click()
time.sleep(1)

# 5-2. 브라우저 등록안함 버튼 클릭
# browser.find_element_by_id('new.dontsave').click()
# time.sleep(1)

# 6. Naver blog 페이지 들어가기
end = False
category_list_old = pd.DataFrame()
category_list_new = pd.DataFrame()
category_list_df = pd.DataFrame()
currentPage = 1
while tqdm((end == False)): 
    browser.get(f"https://blog.naver.com/PostTitleListAsync.naver?blogId={blogId}&viewdate=&currentPage={currentPage}&categoryNo={categoryNo}&parentCategoryNo=&countPerPage=30")
    html = browser.page_source
    time.sleep(1)
    soup = bs(html, 'html.parser')
    body_list = soup.find('body').get_text().replace('[', ']').split(']')
    body = body_list[1].replace('false', 'False').replace('true', 'True').replace('none', 'None')
    category_list = eval('['+body+']')
    if(currentPage == 1):
        category_list_new = category_list
        category_list_df = pd.DataFrame(category_list_new)
    else:
        category_list_new = category_list
        if(category_list_old == category_list_new):
            end = True
        else:
            category_list_df = pd.concat([category_list_df,  pd.DataFrame(category_list_new)])
    category_list_old = category_list_new
    currentPage += 1
    category_list_df = category_list_df.reset_index(drop=True)
category_list_df.to_csv(f'category_list({blogId}_{categoryNo}_{len(category_list_df)}).csv', index=False)    
print(f'{blogId}님의 블로그 게시물 리스트 수집이 완료되었습니다.')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

yyyybbom님의 블로그 게시물 리스트 수집이 완료되었습니다.


In [ ]:
category_list_df

,sellerServiceStatus,logNo,title,categoryNo,parentCategoryNo,sourceCode,commentCount,readCount,addDate,openType,searchYn,greenReviewBannerYn,memologMovingYn,isPostSelectable,isPostNotOpen,isPostBlocked,isBlockTmpForced,postProductStatus
0,,222848608839,%23%EC%A0%9C%EC%A3%BC%EA%B3%B5%ED%95%AD+5%EB%B...,10,1,0,,,2022. 8. 15.,2,True,False,0,0,0,0,0,
1,,222839537245,%23%EC%A0%9C%EC%A3%BC%EC%84%B8%ED%99%94%EC%B9%...,10,1,0,,,2022. 8. 4.,2,True,False,0,0,0,0,0,
2,,222839519420,%23%EC%98%A4%EB%8A%98%EB%8F%84%EC%A0%9C%EC%A3%...,10,1,0,,,2022. 8. 4.,2,True,False,0,0,0,0,0,
3,,222832653450,%23%EC%A0%9C%EC%A3%BC%ED%95%9C%EA%B2%BD%ED%98%...,10,1,0,,,2022. 7. 29.,2,True,False,0,0,0,0,0,
4,,222832683314,%23%EC%A0%9C%EC%A3%BC%ED%95%9C%EA%B2%BD%EB%A9%...,10,1,0,,,2022. 7. 28.,2,True,False,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,,221213812005,%23%EC%84%9C%EC%9A%B8%EB%8D%B0%EC%9D%B4%ED%8A%...,9,1,0,3,,2018. 2. 22.,2,True,False,0,0,0,0,0,
328,,221213681585,%23%EC%84%9C%EC%9A%B8%EB%8D%B0%EC%9D%B4%ED%8A%...,9,1,0,1,,2018. 2. 22.,2,True,False,0,0,0,0,0,
329,,221213638344,%23%EA%B5%AD%EB%82%B4%EC%97%AC%ED%96%89_+%EA%B...,9,1,0,4,,2018. 2. 21.,2,True,False,0,0,0,0,0,
330,,221213615519,%23%EA%B5%AD%EB%82%B4%EC%97%AC%ED%96%89_%EA%B0...,9,1,0,1,,2018. 2. 21.,2,True,False,0,0,0,0,0,


## Category_list Datetime 전처리

In [ ]:
category_list_df['addDate'] = pd.to_datetime(category_list_df['addDate'])

In [ ]:
category_list_df

,sellerServiceStatus,logNo,title,categoryNo,parentCategoryNo,sourceCode,commentCount,readCount,addDate,openType,searchYn,greenReviewBannerYn,memologMovingYn,isPostSelectable,isPostNotOpen,isPostBlocked,isBlockTmpForced,postProductStatus
0,,222848608839,%23%EC%A0%9C%EC%A3%BC%EA%B3%B5%ED%95%AD+5%EB%B...,10,1,0,,,2022-08-15,2,True,False,0,0,0,0,0,
1,,222839537245,%23%EC%A0%9C%EC%A3%BC%EC%84%B8%ED%99%94%EC%B9%...,10,1,0,,,2022-08-04,2,True,False,0,0,0,0,0,
2,,222839519420,%23%EC%98%A4%EB%8A%98%EB%8F%84%EC%A0%9C%EC%A3%...,10,1,0,,,2022-08-04,2,True,False,0,0,0,0,0,
3,,222832653450,%23%EC%A0%9C%EC%A3%BC%ED%95%9C%EA%B2%BD%ED%98%...,10,1,0,,,2022-07-29,2,True,False,0,0,0,0,0,
4,,222832683314,%23%EC%A0%9C%EC%A3%BC%ED%95%9C%EA%B2%BD%EB%A9%...,10,1,0,,,2022-07-28,2,True,False,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,,221213812005,%23%EC%84%9C%EC%9A%B8%EB%8D%B0%EC%9D%B4%ED%8A%...,9,1,0,3,,2018-02-22,2,True,False,0,0,0,0,0,
328,,221213681585,%23%EC%84%9C%EC%9A%B8%EB%8D%B0%EC%9D%B4%ED%8A%...,9,1,0,1,,2018-02-22,2,True,False,0,0,0,0,0,
329,,221213638344,%23%EA%B5%AD%EB%82%B4%EC%97%AC%ED%96%89_+%EA%B...,9,1,0,4,,2018-02-21,2,True,False,0,0,0,0,0,
330,,221213615519,%23%EA%B5%AD%EB%82%B4%EC%97%AC%ED%96%89_%EA%B0...,9,1,0,1,,2018-02-21,2,True,False,0,0,0,0,0,


## Potsts logNo 리스트 변환

In [ ]:
post_list = cagegory_list_df['logNo'].tolist()

# 네이버 블로그 게시물 내용 수집 

## 게시물 내용 수집(requests) 테스트

In [ ]:
# 블로그ID/게시물번호
blogId = '블로그ID'
currentPage = 1
categoryNo = 0
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36",
}
resp = requests.get(f"https://m.blog.naver.com/{blogId}/222298001545", headers=headers)
soup = bs(resp.text, 'html.parser')
post = soup.find(id="_post_area").get_text(strip=True, separator="\n")
post

'대표 성장노트\n강의오답노트 #1 학생의 말을 있는 그대로 신뢰할 것, 툴은 완벽히 파악한 후 적용할 것\n강지연\n2021. 4. 4. 11:44\n이웃추가\n본문 기타 기능\n본문 폰트 크기 조정\n본문 폰트 크기 작게 보기\n본문 폰트 크기 크게 보기\n가\n공감하기\n공유하기\nURL복사\n신고하기\n\u200b\n어제 수업은 이상했다. 보통은 끝난 직후 무언가 잘 되었다 그렇지 않았다 라는 판단이 서는데 어제의 것은 말 그대로 이상했다. 집으로 돌아와 참여자들의 실습 과제를 모두 다시 보며 편집* 할 때에야 이상한 것이 무엇인지 깨달을 수 있었다.\n\u200b\n\u200b\n1. 그들은 정말로 순수하게 그저 좋아서 하는 사람들이었다.\n이번 강의의 타이틀은 \'마음친구* 활동을 내 것으로 만드는 방법\' 이다. 마음친구는 서울시청년활동지원센터의 온라인 고민상담소 hi, there에서 활동하는 또래 고민 상담사 들이다. 약 한달 간 교육을 받은 뒤 마음친구로 활동한다. 내가 수업했던 날은 마지막 교육 날 이었다. 내가 맡은 건 지난 한달 간의 교육 회고 및 앞으로의 활동에 동기부여였다. 담당자님이 특별히 부탁한 부분이 본 활동을 개인적 경험으로 자산화 하는 방법을 알려달라는 것이었다. 마음친구로서 잘 하는 것도 중요하지만 이것이 후에 개인의 자산이 되어 다른 일을 할 때도 도움이 될 수 있도록 방법을 알려주고 관련하여 메시지를 달라는 것이었다.\n\u200b\n그렇다. 그래서 나를 섭외한 것이었다. \'좋아하는 일을 업으로 만드는 방법\' 강의를 하니까 뭐든 자기 것으로 잘 만드는 방법을 알려주리라 기대 한 것 이었다.\n\u200b\n그러나 수업 준비 내내 난항이었다. 자산화가 되려면 그들이 궁극에 도달하고자 하는 목적지가 무엇인지 알아야 하는데 도무지 그것이 보이지 않았다. 심리 상담사로서 경험을 쌓고자 하는 것인지, 자신의 적성이 해당 방향이 맞는지 확인하고 싶은 것인지 등등. 사전에 자소서와 이력서를 받아 보았다. 두 번이나 다시 보았지만 

## 게시물 내용 수집(selenium) 테스트

### 데이터 전처리

In [ ]:
import re

def is_valid_format_date(_date) :
    regex = r'(\d{4}. \d{1,2}. \d{1,2}. \d{1,2}:\d{1,2})'
    return  bool(re.match(regex, _date))

In [ ]:
def post_preprocessing(post: 'str') -> dict:
    post_dict = {}
    post_split = post.split('\n')
    idx_d = 0
    idx_f = 0
    idx_b = 0
    idx_a = 0
    idx_t = 0
    for idx,text in enumerate(post_split):
        if idx_d == 0:
            if is_valid_format_date(text):
                idx_d = idx
        if text == '신고하기':
            idx_f = idx + 1
        if text == '공감한 사람 보러가기': 
            idx_a = idx
        if idx_t == 0:
            if text == '댓글':
                idx_t = idx
    if idx_a > 0:
        idx_b = idx_t - 2
    else:
        idx_b = idx_t - 1
        
    blog_category = post_split[0]
    posttitle = " ".join(post_split[1 : idx_d - 1])
    nickname = post_split[idx_d - 1]
    blog_date = post_split[idx_d]
    blog_btnArea = post_split[idx_d + 1]
    post_text_list = post_split[idx_f : idx_b]
    post_text = ''
    for idx, text in enumerate(post_text_list):
        if text == '\u200b':
            pass
        else:
            post_text = post_text + '\n' + text
    post_dict['category'] = blog_category
    post_dict['posttitle'] = posttitle
    post_dict['nickname'] = nickname
    post_dict['date'] = blog_date
    post_dict['btnArea'] = blog_btnArea
    post_dict['text'] = post_text
    return post_dict

## 게시물 내용 수집(selenium) 

In [ ]:
# 블로그ID/게시물번호
blogId = '블로그ID'
currentPage = 1
categoryNo = 0
post_list = pd.read_csv(f'category_list({blogId}_{categoryNo}).csv')['logNo'].tolist()
post_list = post_list[:50]

In [ ]:
# 0-1. 브라우저 안띄우기
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# browser = webdriver.Chrome('chromedriver.exe', chrome_options=options)

# 0-2. 브라우저 띄우기
browser = webdriver.Chrome('chromedriver.exe') # 현재파일과 동일한 경로일 경우 생략 가능

user_id = '네이버ID'
user_pw = '네이버PW'

# 1. 네이버 이동
browser.get('http://naver.com')

# 2. 로그인 버튼 클릭
elem = browser.find_element_by_class_name('link_login')
elem.click()

# 3. id 복사 붙여넣기
elem_id = browser.find_element_by_id('id')
elem_id.click()
pyperclip.copy(user_id)
elem_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 4. pw 복사 붙여넣기
elem_pw = browser.find_element_by_id('pw')
elem_pw.click()
pyperclip.copy(user_pw)
elem_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 5-1. 로그인 버튼 클릭
browser.find_element_by_id('log.login').click()
time.sleep(1)

# 5-2. 브라우저 등록안함 버튼 클릭
# browser.find_element_by_id('new.dontsave').click()
# time.sleep(1)

post_df = pd.DataFrame()
for idx, logNo in enumerate(tqdm(post_list)):
    # Naver 페이 들어가기
    browser.get(f'https://m.blog.naver.com/{blogId}/{logNo}')
    html = browser.page_source
    soup = bs(html, 'html.parser')
    post = soup.find(id="_post_area").get_text(strip=True, separator="\n")
    
    if idx == 0:
        post_df = pd.DataFrame([post_preprocessing(post)])
        post_df['logNo'] = logNo
    else:
        post_df_new = pd.DataFrame([post_preprocessing(post)])
        post_df_new['logNo'] = logNo
        post_df = pd.concat([post_df, post_df_new])
    post_df = post_df.reset_index(drop=True)
    post_df.to_csv(f'posts({blogId}_{categoryNo}_{len(post_list)}).csv', index=False)

100%|██████████| 50/50 [01:57<00:00,  2.35s/it]


In [ ]:
test_df = pd.read_csv(f'posts({blogId}_{categoryNo}_{len(post_list)}).csv')

In [ ]:
test_df

,category,posttitle,nickname,date,btnArea,text,logNo
0,청년진로문답,남들쌓는 스펙 다 쌓고보니 어영부영된 느낌...? 취직할 때 어중이떠중이 될까봐 걱...,강지연,2021. 4. 7. 21:53,서로이웃,\n그럴 수 있죠. 그래도 대단한 건 '행동'하셨다는 거예요. 생각보다 많은 사람들...,222302271153
1,대표 성장노트,어제는 일부러 수업을 쉬었어요.,강지연,2021. 4. 6. 21:26,서로이웃,"\n어제는 <좋아하는 일을 업으로 하는 방법> 정규 수업 날이었는데요, 일부러 하루...",222300959223
2,대표 성장노트,"강의오답노트 #1 학생의 말을 있는 그대로 신뢰할 것, 툴은 완벽히 파악한 후 적용할 것",강지연,2021. 4. 4. 11:44,서로이웃,\n어제 수업은 이상했다. 보통은 끝난 직후 무언가 잘 되었다 그렇지 않았다 라는 ...,222298001545
3,청년진로문답,취업할 때 대외활동이 많이 중요한가요?,강지연,2021. 3. 31. 20:10,서로이웃,"\n진로, 취업 특강에서 가장 많이 받는 질문인데요. 결론부터 이야기하면 네, 인사...",222294125712
4,대표 성장노트,먼저 더 많은 불편을 감당해야겠단 생각을 했다.,강지연,2021. 3. 31. 7:38,서로이웃,\n아침에 눈을 떠보니 장문의 카톡이 와있었다. 정규과정을 듣고 있는 소정님의 메시...,222293445237
5,대표 성장노트,자신을 견디기 위해선 좋은 이름이 필요해,강지연,2021. 3. 31. 0:01,서로이웃,\n1. 지난 달 수업을 들었던 A가 한 달만에 큰 성장을 이뤘다. 가족들에게조차 ...,222293226481
6,청년진로문답,다들 목표를 향해 달려가는 것 같은데 저만 아직 꿈이 없어요,강지연,2021. 3. 30. 8:07,서로이웃,"\n꿈이 꼭 있어야 하나요?\n세계적인 심리학자, 엔절라 더크워스의 저서 GRIT에...",222292239539
7,청년진로문답,한 분야에서 특출나기 보단 여러 분야에서 애매하게 재능이 있어 어디로 가야할지 모르...,강지연,2021. 3. 30. 8:00,서로이웃,"\n어쩌면 그것이 바로 A님만의 특출난 점은 아닐까요?\n미국의 저명한 심리학자, ...",222292235825
8,좋아하는일을업으로만드는방법,목표가 흔들릴 때 가장 먼저 던져야 할 질문,강지연,2021. 3. 29. 11:24,서로이웃,\n10년 간 스튜어디스만 보고 달려온 J였다. 요즘 부쩍 불안하다 말했다. 정말로...,222291233819
9,대표 성장노트,해야 할 일은 언젠가 다시 하게 된다,강지연,2021. 3. 29. 5:49,서로이웃,\n6년 전 기획서를 다시 꺼내보았다. 내가 할 일이 결국 이것이구나 하는 마음으로...,222291025071
